In [14]:
%%time
import pandas as pd
import numpy as np
import gensim
from gensim.models import CoherenceModel
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from statistics import mean
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance
from gensim.matutils import softcossim
import collections
import math
import nltk
lemma = nltk.wordnet.WordNetLemmatizer()
#python -m pip install -U pycld2
import pycld2 as cld2
import spacy
sp = spacy.load("en")
sp_stopwords=sp.Defaults.stop_words
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline
import gensim.downloader as api
from gensim import corpora,models
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
w2v_model = api.load("glove-wiki-gigaword-50")

/Users/Lukre/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CPU times: user 26.4 s, sys: 536 ms, total: 26.9 s
Wall time: 27.4 s


Učitavanje dokumenata

In [15]:
doc_risk_str=pd.read_csv('/Users/Lukre/Desktop/D_dipl/doc_risk_str.csv',index_col=0)
doc_risk_tok=pd.read_csv('/Users/Lukre/Desktop/D_dipl/doc_risk_tok.csv',index_col=0)
for i in range(len(doc_risk_tok)):
    doc_risk_tok.report[i]=str(doc_risk_str.report[i]).split()

doc_no_risk_str=pd.read_csv('/Users/Lukre/Desktop/D_dipl/doc_no_risk_str.csv',index_col=0)
doc_no_risk_tok=pd.read_csv('/Users/Lukre/Desktop/D_dipl/doc_no_risk_tok.csv',index_col=0)
for i in range(len(doc_no_risk_tok)):
    doc_no_risk_tok.report[i]=str(doc_no_risk_str.report[i]).split()

pdoc_risk=pd.read_csv('/Users/Lukre/Desktop/D_dipl/pdoc_risk.csv',index_col=0)
tokens_risk=pd.read_csv('/Users/Lukre/Desktop/D_dipl/tokens_risk.csv',index_col=0)
pdoc_no_risk=pd.read_csv('/Users/Lukre/Desktop/D_dipl/pdoc_no_risk.csv',index_col=0)
tokens_no_risk=pd.read_csv('/Users/Lukre/Desktop/D_dipl/tokens_no_risk.csv',index_col=0)
sparse_risk=pd.read_csv('/Users/Lukre/Desktop/D_dipl/sparse_risk.csv',index_col=0)
sparse_no_risk=pd.read_csv('/Users/Lukre/Desktop/D_dipl/sparse_no_risk.csv',index_col=0)
pdoc_risk_red=pd.read_csv('/Users/Lukre/Desktop/D_dipl/pdoc_risk_red.csv',index_col=0)
tokens_risk_red=pd.read_csv('/Users/Lukre/Desktop/D_dipl/tokens_risk_red.csv',index_col=0)
ejected_tokens_risk=pd.read_csv('/Users/Lukre/Desktop/D_dipl/ejected_tokens_risk.csv',index_col=0)
pdoc_no_risk_red=pd.read_csv('/Users/Lukre/Desktop/D_dipl/pdoc_no_risk_red.csv',index_col=0)
tokens_no_risk_red=pd.read_csv('/Users/Lukre/Desktop/D_dipl/tokens_no_risk_red.csv',index_col=0)
ejected_tokens_no_risk=pd.read_csv('/Users/Lukre/Desktop/D_dipl/ejected_tokens_no_risk.csv',index_col=0)
sparse_risk_red=pd.read_csv('/Users/Lukre/Desktop/D_dipl/sparse_risk_red.csv',index_col=0)
sparse_no_risk_red=pd.read_csv('/Users/Lukre/Desktop/D_dipl/sparse_no_risk_red.csv',index_col=0)

/Users/Lukre/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/Lukre/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/Lukre/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

Fja vraća 5 najsličnijih dokumenata po cosinusu (vraća: ID i sličnost)

In [17]:
def five_most_similar(xx,sparse):
    xx_5=[0, 1, 2, 3, 4]
    maxi=[0,0,0,0,0]
    for row in sparse.index:
        if (row[0]==xx):
            continue
        pom=distance.cosine(sparse.loc[xx],sparse.loc[row[0]])
        pom=1-pom
        for j in range(5):
            if pom > maxi[j]:
                maxi[j]=pom
                xx_5[j]=row[0]
                break
    return xx_5, maxi  

/Users/Lukre/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Fja prima:  
&emsp;-tokenizirani dokument s izvještajima (prep_doc(risk,tok)) rijetku matricu matricu i oznaku 'cos' za Cosine similarity i proizvoljni string za SoftCosine similarity  
  
  Fja vraća:  
&emsp;-konfiguracijsku matricu i evaluacijske vrijednosti: Precision, Recall i F1 score

In [18]:
def cos_evaluation(doc_tok,sparse,cosin='cos',rnd=42,label='label'):
    
    if (label=='label'):
        X_train, X_val, y_train, y_val = train_test_split(doc_tok.index.values, 
                                                  doc_tok.label.values, 
                                                  test_size=0.2, 
                                                  random_state=rnd, 
                                                  stratify=doc_tok.label.values)
    elif (label=='accident'):
        X_train, X_val, y_train, y_val = train_test_split(doc_tok.index.values, 
                                                  doc_tok.accident.values, 
                                                  test_size=0.2, 
                                                  random_state=rnd, 
                                                  stratify=doc_tok.accident.values)
    
    doc_tok['data_type'] = ['not_set']*doc_tok.shape[0]

    doc_tok.loc[X_train, 'data_type'] = 'train'
    doc_tok.loc[X_val, 'data_type'] = 'val'

    if (label=='label'):
        doc_tok.groupby(['label', 'data_type']).count()
    elif (label=='accident'):
        doc_tok.groupby(['accident','data_type']).count()

    CosSim_results=pd.DataFrame(columns=['Precision','Recall','F1-score'],index=['Value'])
    zero = np.zeros(shape=(3,3))
    if (label=='label'):
        conf_matrix=pd.DataFrame(zero,columns=[0,1,2],index=[0,1,2])
    elif (label=='accident'):
        conf_matrix=pd.DataFrame(zero,columns=['no accident outcome','minor injuries or damage',
                                              'major or catastrophic accident'],
                                index=['no accident outcome','minor injuries or damage',
                                              'major or catastrophic accident'])
    
    if (cosin!='cos'):
        sentences = [x for x in doc_tok.report]
        dictionary = corpora.Dictionary(sentences)
        similarity_index = WordEmbeddingSimilarityIndex(w2v_model)
        similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary)
        for i in range(len(sentences)):
            sentences[i]=dictionary.doc2bow(sentences[i])

    for i in range(len(X_val)):
        maxi=0
        pomj=0
        
        for j in range(len(X_train)):
            if (cosin=='cos'):
                pom=distance.cosine(sparse.loc[doc_tok.xx[X_val[i]]],sparse.loc[doc_tok.xx[X_train[j]]])
                pom=1-pom
            else: 
                pom=similarity_matrix.inner_product(sentences[X_val[i]], sentences[X_train[j]], normalized=True)
            if pom > maxi:
                maxi=pom
                pomj=j
        conf_matrix.loc[y_train[j]][y_val[i]]+=1
        if (i%10==0):
            print(i)
            print (maxi)
    
    if (label=='label'):
        p0=conf_matrix.loc[0][0]/(sum(conf_matrix.loc[0])+1)
        p1=conf_matrix.loc[0][1]/(sum(conf_matrix.loc[1])+1)
        p2=conf_matrix.loc[2][2]/(sum(conf_matrix.loc[2])+1)
        r0=conf_matrix.loc[0][0]/(sum(conf_matrix[0])+1)
        r1=conf_matrix.loc[1][1]/(sum(conf_matrix[1])+1)
        r2=conf_matrix.loc[2][2]/(sum(conf_matrix[2])+1)
        P=mean([p0,p1,p2])
        R=mean([r0,r1,r2])
        F1=(2*P*R)/(P+R)
    elif(label=='accident'):
        p0=conf_matrix.loc['no accident outcome']['no accident outcome']/(sum(conf_matrix.loc['no accident outcome'])+1)
        p1=conf_matrix.loc['minor injuries or damage']['minor injuries or damage']/(sum(conf_matrix.loc['minor injuries or damage'])+1)
        p2=conf_matrix.loc['major or catastrophic accident']['major or catastrophic accident']/(sum(conf_matrix.loc['major or catastrophic accident'])+1)
        r0=conf_matrix.loc['no accident outcome']['no accident outcome']/(sum(conf_matrix['no accident outcome'])+1)
        r1=conf_matrix.loc['minor injuries or damage']['minor injuries or damage']/(sum(conf_matrix['minor injuries or damage'])+1)
        r2=conf_matrix.loc['major or catastrophic accident']['major or catastrophic accident']/(sum(conf_matrix['major or catastrophic accident'])+1)
        P=mean([p0,p1,p2])
        R=mean([r0,r1,r2])
        F1=(2*P*R)/(P+R)      

    CosSim_results.loc['Value']['Precision']=P
    CosSim_results.loc['Value']['Recall']=R
    CosSim_results.loc['Value']['F1-score']=F1
    
    return (CosSim_results,conf_matrix)

/Users/Lukre/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Evaluacija cosinusom  
&emsp;uvijek predviđa razinu 0  
NE DAJE DOBRE REZULTATE

In [20]:
%%time
cos_evaluation(doc_risk_tok,sparse_risk_red,label='accident')

## ovom metodom predvidi se uvijek razina 0  -nije dobro
# međutim primjetne su veće sličnosti nego prije reduciranja rijetkih tokena

/Users/Lukre/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0
0.30561996406666836
10
0.3266733959487864
20
0.43002538827353165
30
0.41049273665922636
40
0.4102589350432827
50
0.4436050139136293
60
0.3070994385021175
70
0.3703631103054662
80
0.4166178232646125


KeyboardInterrupt: 

Random Forest Classifier    -scikit learn    
&emsp;accident

In [23]:
RFC_results=pd.DataFrame(columns=['Precision','Recall','F1-score'],index=range(10))
for i in range(10):

    zero = np.zeros(shape=(3,3))
    conf_matrix=pd.DataFrame(zero,columns=['no accident outcome','minor injuries or damage',
                                              'major or catastrophic accident'],
                                index=['no accident outcome','minor injuries or damage',
                                              'major or catastrophic accident'])

    X_train, X_val, y_train, y_val = train_test_split(doc_risk_tok.index.values, 
                                                  doc_risk_tok.accident.values, 
                                                  test_size=0.2, 
                                                  random_state=42, 
                                                  stratify=doc_risk_tok.accident.values)
    doc_risk_tok['data_type'] = ['not_set']*doc_risk_tok.shape[0]
    doc_risk_tok.loc[X_train, 'data_type'] = 'train'
    doc_risk_tok.loc[X_val, 'data_type'] = 'val'

    xx_train=doc_risk_tok[doc_risk_tok.data_type=='train'].xx
    xx_val=doc_risk_tok[doc_risk_tok.data_type=='val'].xx
   
    clf = RandomForestClassifier(n_estimators=100, class_weight='balanced', max_features=None)
    #max_features='sqrt'
    clf.fit(sparse_risk.loc[doc_risk_tok[doc_risk_tok.data_type=='train'].xx],doc_risk_tok[doc_risk_tok.data_type=='train'].accident)

    for w in xx_val:
        pom=clf.predict(np.array(sparse_risk.loc[w]).reshape(1,-1))[0]
        conf_matrix.loc[pom][doc_risk_tok[doc_risk_tok.xx==w].accident]+=1
    

    p0=conf_matrix.loc['no accident outcome']['no accident outcome']/(sum(conf_matrix.loc['no accident outcome']))
    p1=conf_matrix.loc['minor injuries or damage']['minor injuries or damage']/(sum(conf_matrix.loc['minor injuries or damage']))
    p2=conf_matrix.loc['major or catastrophic accident']['major or catastrophic accident']/(sum(conf_matrix.loc['major or catastrophic accident']))
    r0=conf_matrix.loc['no accident outcome']['no accident outcome']/(sum(conf_matrix['no accident outcome']))
    r1=conf_matrix.loc['minor injuries or damage']['minor injuries or damage']/(sum(conf_matrix['minor injuries or damage']))
    r2=conf_matrix.loc['major or catastrophic accident']['major or catastrophic accident']/(sum(conf_matrix['major or catastrophic accident']))
    P=mean([p0,p1,p2])
    R=mean([r0,r1,r2])
    F1=(2*P*R)/(P+R)

    RFC_results.loc[i]['Precision']=P
    RFC_results.loc[i]['Recall']=R
    RFC_results.loc[i]['F1-score']=F1

    print(conf_matrix)
    #print(i)
    #print(i+100*i)
print(RFC_results)
RESULTS=pd.DataFrame(columns=['PRECISION','RECALL','F1-SCORE'],index=['Value'])    
RESULTS['PRECISION']=sum(RFC_results['Precision'])/10
RESULTS['RECALL']=sum(RFC_results['Recall'])/10
RESULTS['F1-SCORE']=sum(RFC_results['F1-score'])/10

/Users/Lukre/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                                no accident outcome  minor injuries or damage  \
no accident outcome                            77.0                      78.0   
minor injuries or damage                      103.0                     231.0   
major or catastrophic accident                  4.0                      13.0   

                                major or catastrophic accident  
no accident outcome                                       16.0  
minor injuries or damage                                  43.0  
major or catastrophic accident                             8.0  
                                no accident outcome  minor injuries or damage  \
no accident outcome                            84.0                      78.0   
minor injuries or damage                       97.0                     233.0   
major or catastrophic accident                  3.0                      11.0   

                                major or catastrophic accident  
no accident outcome      

In [11]:
print(RESULTS)  #NEREDUCIRANI

       PRECISION    RECALL  F1-SCORE
Value   0.474987  0.434331  0.453702


/Users/Lukre/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
clf.feature_importances_
sorted_idx = clf.feature_importances_.argsort() #indeksi sortiranog vektora
#sorted_features = clf.feature_importances_.sort #sortirani vektor
ide = [0]*len(sorted_idx)
for i in range (len (sorted_idx)):
    ide[i] = clf.feature_importances_[sorted_idx[i]]
ide2=[0]*10
idx2=[0]*10
for i in range (10):
    idx2[i]=sorted_idx[len(sorted_idx)-1-i]
    ide2[i] = ide[len(sorted_idx)-i-1]
columnnames=['']*10
j=0
for i in idx2:
    columnnames[j]=col_names[i]
    j=j+1
plt.rcParams.update({'figure.figsize': (10.0, 6.0)})
plt.rcParams.update({'font.size': 12})
col_names=list(sparse_risk_red.columns)
plt.barh(columnnames, ide2)
plt.xlabel("Random Forest Feature Importance")

/Users/Lukre/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([ 270, 1194, 1242, ..., 1433,  118,  285])